# FHE Training Demo

This notebook demonstrates Fully Homomorphic Encryption (FHE) training and inference using Concrete-ML.

## Overview
- Load preprocessed medical data
- Train Concrete-ML logistic regression model
- Run encrypted inference on test data
- Compare clear vs encrypted predictions visually
- Display confusion matrix heatmap for analysis

## Key Features
- **Privacy-Preserving ML**: Train models that can operate on encrypted data
- **Performance Comparison**: Clear vs encrypted accuracy analysis
- **Visual Analysis**: Comprehensive plots and confusion matrices
- **Real-world Application**: Medical data classification with privacy protection

---

In [ ]:
# Import required libraries
import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
import time

# Add the src directory to Python path
sys.path.append('../src')

# Try to import Concrete-ML (with fallback)
try:
    from concrete.ml.sklearn import LogisticRegression as FHELogisticRegression
    CONCRETE_ML_AVAILABLE = True
    print("✅ Concrete-ML available for FHE operations")
except ImportError:
    from sklearn.linear_model import LogisticRegression as FHELogisticRegression
    CONCRETE_ML_AVAILABLE = False
    print("⚠️ Concrete-ML not available, using sklearn for demonstration")

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("📚 All imports successful!")
print(f"🔐 FHE Mode: {'Enabled' if CONCRETE_ML_AVAILABLE else 'Simulation'}")

## 1. Load Preprocessed Data

Load the preprocessed medical data for FHE training and evaluation.

In [ ]:
# Load preprocessed data
data_path = Path('../data/processed/preprocessed_data.json')

if data_path.exists():
    print(f"📂 Loading preprocessed data from: {data_path}")
    with open(data_path, 'r', encoding='utf-8') as f:
        preprocessed_data = json.load(f)
    
    records = preprocessed_data.get('processed_records', [])
    print(f"✅ Loaded {len(records)} preprocessed records")
else:
    print("⚠️ Preprocessed data not found. Creating synthetic data for demonstration...")
    
    # Create synthetic preprocessed data
    np.random.seed(42)
    n_samples = 500
    
    conditions = ['diabetes', 'hypertension', 'heart_disease', 'asthma', 'arthritis']
    age_groups = ['18-30', '31-50', '51-65', '65+']
    
    records = []
    for i in range(n_samples):
        condition = np.random.choice(conditions)
        record = {
            'patient_id': f'patient_{i:04d}',
            'age_group': np.random.choice(age_groups),
            'medical_notes_processed': f"Patient presents with {condition} symptoms and related complications",
            'primary_condition': condition,
            'risk_score': np.random.uniform(0.1, 0.9),
            'anonymized': True
        }
        records.append(record)
    
    print(f"✅ Created {len(records)} synthetic records for demonstration")

# Convert to DataFrame
df = pd.DataFrame(records)
print(f"📊 DataFrame shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")

# Display basic info
print("\n📈 Data Overview:")
print(df.head())

## 2. Data Preprocessing for ML

Prepare the data for machine learning by extracting features and encoding labels.

In [ ]:
# Prepare features and labels
print("🔧 Preparing features and labels for ML training...")

# Extract text features using TF-IDF
print("📝 Extracting text features from medical notes...")
vectorizer = TfidfVectorizer(
    max_features=100,  # Limit features for FHE compatibility
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2
)

# Vectorize medical notes
X_text = vectorizer.fit_transform(df['medical_notes_processed']).toarray()
print(f"📊 Text features shape: {X_text.shape}")

# Add numerical features if available
numerical_features = []
if 'risk_score' in df.columns:
    numerical_features.append(df['risk_score'].values.reshape(-1, 1))
    print("📈 Added risk score as numerical feature")

# Encode age groups as numerical features
if 'age_group' in df.columns:
    age_encoder = LabelEncoder()
    age_encoded = age_encoder.fit_transform(df['age_group']).reshape(-1, 1)
    numerical_features.append(age_encoded)
    print("🎂 Added encoded age group as numerical feature")

# Combine all features
if numerical_features:
    X_numerical = np.hstack(numerical_features)
    X = np.hstack([X_text, X_numerical])
    print(f"🔗 Combined features shape: {X.shape}")
else:
    X = X_text
    print(f"📊 Using text features only: {X.shape}")

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['primary_condition'])

print(f"🏷️ Labels encoded: {len(label_encoder.classes_)} classes")
print(f"📋 Classes: {list(label_encoder.classes_)}")
print(f"🎯 Label distribution:")
for i, class_name in enumerate(label_encoder.classes_):
    count = np.sum(y == i)
    print(f"   {class_name}: {count} samples ({count/len(y)*100:.1f}%)")

# Scale features for better FHE performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"⚖️ Features scaled to mean=0, std=1")
print(f"📊 Final feature matrix shape: {X_scaled.shape}")
print(f"🎯 Final label vector shape: {y.shape}")

## 3. Train-Test Split

Split the data into training and testing sets for model evaluation.

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
)

print("📊 Data Split Summary:")
print(f"   Training set: {X_train.shape[0]} samples")
print(f"   Test set: {X_test.shape[0]} samples")
print(f"   Features: {X_train.shape[1]} dimensions")
print(f"   Classes: {len(label_encoder.classes_)}")

# Display class distribution in train/test sets
print("\n🎯 Training Set Distribution:")
train_unique, train_counts = np.unique(y_train, return_counts=True)
for class_idx, count in zip(train_unique, train_counts):
    class_name = label_encoder.classes_[class_idx]
    print(f"   {class_name}: {count} samples ({count/len(y_train)*100:.1f}%)")

print("\n🎯 Test Set Distribution:")
test_unique, test_counts = np.unique(y_test, return_counts=True)
for class_idx, count in zip(test_unique, test_counts):
    class_name = label_encoder.classes_[class_idx]
    print(f"   {class_name}: {count} samples ({count/len(y_test)*100:.1f}%)")

print(f"\n✅ Data preparation completed successfully!")

## 4. Train Clear (Non-Encrypted) Model

First, train a standard logistic regression model for baseline comparison.

In [ ]:
# Train clear (non-encrypted) logistic regression model
print("🔓 Training Clear Logistic Regression Model...")
print("=" * 50)

start_time = time.time()

# Initialize and train clear model
clear_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    multi_class='ovr'  # One-vs-Rest for multi-class
)

clear_model.fit(X_train, y_train)
clear_training_time = time.time() - start_time

print(f"✅ Clear model training completed in {clear_training_time:.2f} seconds")

# Make predictions on test set
start_time = time.time()
y_pred_clear = clear_model.predict(X_test)
clear_inference_time = time.time() - start_time

# Calculate accuracy
clear_accuracy = accuracy_score(y_test, y_pred_clear)

print(f"🎯 Clear Model Performance:")
print(f"   Training time: {clear_training_time:.2f} seconds")
print(f"   Inference time: {clear_inference_time:.4f} seconds")
print(f"   Test accuracy: {clear_accuracy:.4f} ({clear_accuracy*100:.2f}%)")

# Display detailed classification report
print(f"\n📊 Detailed Classification Report (Clear Model):")
class_names = label_encoder.classes_
print(classification_report(y_test, y_pred_clear, target_names=class_names, digits=4))

## 5. Train Concrete-ML FHE Model

Train the FHE-compatible logistic regression model using Concrete-ML.

In [ ]:
# Train FHE-compatible logistic regression model
print("🔐 Training FHE Logistic Regression Model...")
print("=" * 50)

start_time = time.time()

# Initialize FHE model with appropriate parameters
if CONCRETE_ML_AVAILABLE:
    fhe_model = FHELogisticRegression(
        n_bits=8,  # Quantization bits for FHE compatibility
        random_state=42,
        max_iter=100  # Reduced for FHE efficiency
    )
    print("🔐 Using Concrete-ML FHE Logistic Regression")
else:
    fhe_model = FHELogisticRegression(
        random_state=42,
        max_iter=1000,
        multi_class='ovr'
    )
    print("⚠️ Using sklearn Logistic Regression (simulation mode)")

# Train the FHE model
fhe_model.fit(X_train, y_train)
fhe_training_time = time.time() - start_time

print(f"✅ FHE model training completed in {fhe_training_time:.2f} seconds")

# Compile the model for FHE inference (if Concrete-ML is available)
if CONCRETE_ML_AVAILABLE:
    print("🔧 Compiling model for FHE inference...")
    start_time = time.time()
    
    try:
        # Use a subset of training data for compilation
        X_compile = X_train[:100]  # Use smaller subset for faster compilation
        fhe_model.compile(X_compile)
        compilation_time = time.time() - start_time
        print(f"✅ FHE compilation completed in {compilation_time:.2f} seconds")
        FHE_COMPILED = True
    except Exception as e:
        print(f"⚠️ FHE compilation failed: {e}")
        print("📝 Continuing with simulation mode...")
        FHE_COMPILED = False
        compilation_time = 0
else:
    FHE_COMPILED = False
    compilation_time = 0

print(f"\n📊 FHE Model Training Summary:")
print(f"   Training time: {fhe_training_time:.2f} seconds")
print(f"   Compilation time: {compilation_time:.2f} seconds")
print(f"   FHE mode: {'Enabled' if FHE_COMPILED else 'Simulation'}")

## 6. Run Encrypted Inference

Perform inference on encrypted test data and compare with clear predictions.

In [ ]:
# Run encrypted inference
print("🔐 Running Encrypted Inference...")
print("=" * 50)

start_time = time.time()

if FHE_COMPILED and CONCRETE_ML_AVAILABLE:
    print("🔐 Performing true FHE encrypted inference...")
    
    # Use a smaller subset for encrypted inference (due to computational cost)
    n_encrypted_samples = min(50, len(X_test))
    X_test_encrypted = X_test[:n_encrypted_samples]
    y_test_encrypted = y_test[:n_encrypted_samples]
    
    print(f"📊 Running encrypted inference on {n_encrypted_samples} samples...")
    
    try:
        # Perform encrypted inference
        y_pred_encrypted = fhe_model.predict(X_test_encrypted, fhe="execute")
        fhe_inference_time = time.time() - start_time
        
        print(f"✅ Encrypted inference completed in {fhe_inference_time:.2f} seconds")
        print(f"⚡ Average time per sample: {fhe_inference_time/n_encrypted_samples:.4f} seconds")
        
        # Calculate encrypted accuracy
        encrypted_accuracy = accuracy_score(y_test_encrypted, y_pred_encrypted)
        
        # Also get clear predictions for the same subset for comparison
        y_pred_clear_subset = clear_model.predict(X_test_encrypted)
        clear_subset_accuracy = accuracy_score(y_test_encrypted, y_pred_clear_subset)
        
        print(f"\n🎯 Encrypted Inference Results:")
        print(f"   Samples processed: {n_encrypted_samples}")
        print(f"   Encrypted accuracy: {encrypted_accuracy:.4f} ({encrypted_accuracy*100:.2f}%)")
        print(f"   Clear accuracy (same subset): {clear_subset_accuracy:.4f} ({clear_subset_accuracy*100:.2f}%)")
        print(f"   Accuracy difference: {abs(encrypted_accuracy - clear_subset_accuracy):.4f}")
        
        ENCRYPTED_INFERENCE_SUCCESS = True
        
    except Exception as e:
        print(f"❌ Encrypted inference failed: {e}")
        print("📝 Falling back to simulation mode...")
        ENCRYPTED_INFERENCE_SUCCESS = False

else:
    print("⚠️ Running simulation mode (clear inference)...")
    ENCRYPTED_INFERENCE_SUCCESS = False

# If encrypted inference failed or not available, use clear inference for comparison
if not ENCRYPTED_INFERENCE_SUCCESS:
    print("🔄 Using FHE model in simulation mode...")
    
    # Use the FHE model but in clear mode
    y_pred_fhe_clear = fhe_model.predict(X_test)
    fhe_clear_inference_time = time.time() - start_time
    
    # Calculate accuracy
    fhe_clear_accuracy = accuracy_score(y_test, y_pred_fhe_clear)
    
    print(f"✅ FHE model (simulation) inference completed in {fhe_clear_inference_time:.4f} seconds")
    print(f"🎯 FHE model (simulation) accuracy: {fhe_clear_accuracy:.4f} ({fhe_clear_accuracy*100:.2f}%)")
    
    # Use these predictions for comparison
    y_pred_encrypted = y_pred_fhe_clear
    encrypted_accuracy = fhe_clear_accuracy
    n_encrypted_samples = len(X_test)

print(f"\n📊 Inference Performance Summary:")
if ENCRYPTED_INFERENCE_SUCCESS:
    print(f"   Mode: True FHE Encryption")
    print(f"   Samples: {n_encrypted_samples}")
    print(f"   Time: {fhe_inference_time:.2f} seconds")
    print(f"   Speed: {n_encrypted_samples/fhe_inference_time:.2f} samples/second")
else:
    print(f"   Mode: Simulation")
    print(f"   Samples: {n_encrypted_samples}")
    print(f"   Time: {fhe_clear_inference_time:.4f} seconds")
    print(f"   Speed: {n_encrypted_samples/fhe_clear_inference_time:.0f} samples/second")

## 7. Visual Comparison: Clear vs Encrypted Predictions

Compare the predictions from clear and encrypted models visually.

In [ ]:
# Create visual comparison of clear vs encrypted predictions
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Clear vs Encrypted Model Comparison', fontsize=16, fontweight='bold')

# Prepare data for comparison
if ENCRYPTED_INFERENCE_SUCCESS:
    # Use the subset that was actually encrypted
    comparison_true = y_test_encrypted
    comparison_clear = y_pred_clear_subset
    comparison_encrypted = y_pred_encrypted
    comparison_samples = n_encrypted_samples
else:
    # Use full test set for simulation comparison
    comparison_true = y_test
    comparison_clear = y_pred_clear
    comparison_encrypted = y_pred_encrypted
    comparison_samples = len(y_test)

# 1. Accuracy Comparison Bar Chart
models = ['Clear Model', 'FHE Model']
accuracies = [accuracy_score(comparison_true, comparison_clear), 
              accuracy_score(comparison_true, comparison_encrypted)]

bars = ax1.bar(models, accuracies, color=['skyblue', 'lightcoral'], alpha=0.8, edgecolor='black')
ax1.set_title('Model Accuracy Comparison', fontweight='bold')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0, 1)
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. Prediction Agreement Analysis
agreement = (comparison_clear == comparison_encrypted)
agreement_rate = np.mean(agreement)

agreement_labels = ['Agree', 'Disagree']
agreement_counts = [np.sum(agreement), np.sum(~agreement)]
colors = ['lightgreen', 'lightcoral']

wedges, texts, autotexts = ax2.pie(agreement_counts, labels=agreement_labels, 
                                   autopct='%1.1f%%', colors=colors, startangle=90)
ax2.set_title(f'Prediction Agreement\n({agreement_rate:.1%} agreement)', fontweight='bold')

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# 3. Class-wise Accuracy Comparison
class_accuracies_clear = []
class_accuracies_encrypted = []
class_names = label_encoder.classes_

for i, class_name in enumerate(class_names):
    class_mask = (comparison_true == i)
    if np.sum(class_mask) > 0:
        clear_class_acc = accuracy_score(comparison_true[class_mask], comparison_clear[class_mask])
        encrypted_class_acc = accuracy_score(comparison_true[class_mask], comparison_encrypted[class_mask])
    else:
        clear_class_acc = 0
        encrypted_class_acc = 0
    
    class_accuracies_clear.append(clear_class_acc)
    class_accuracies_encrypted.append(encrypted_class_acc)

x_pos = np.arange(len(class_names))
width = 0.35

bars1 = ax3.bar(x_pos - width/2, class_accuracies_clear, width, 
                label='Clear Model', color='skyblue', alpha=0.8)
bars2 = ax3.bar(x_pos + width/2, class_accuracies_encrypted, width,
                label='FHE Model', color='lightcoral', alpha=0.8)

ax3.set_title('Class-wise Accuracy Comparison', fontweight='bold')
ax3.set_xlabel('Medical Conditions')
ax3.set_ylabel('Accuracy')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(class_names, rotation=45, ha='right')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# 4. Sample Predictions Visualization
sample_indices = np.arange(min(20, comparison_samples))
sample_true = comparison_true[:len(sample_indices)]
sample_clear = comparison_clear[:len(sample_indices)]
sample_encrypted = comparison_encrypted[:len(sample_indices)]

# Create a scatter plot showing prediction differences
ax4.scatter(sample_indices, sample_true, label='True Labels', 
           marker='o', s=100, alpha=0.7, color='green')
ax4.scatter(sample_indices, sample_clear, label='Clear Predictions', 
           marker='^', s=80, alpha=0.7, color='blue')
ax4.scatter(sample_indices, sample_encrypted, label='FHE Predictions', 
           marker='s', s=80, alpha=0.7, color='red')

ax4.set_title(f'Sample Predictions Comparison\n(First {len(sample_indices)} samples)', fontweight='bold')
ax4.set_xlabel('Sample Index')
ax4.set_ylabel('Predicted Class')
ax4.set_yticks(range(len(class_names)))
ax4.set_yticklabels(class_names)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed comparison statistics
print("📊 Detailed Comparison Statistics:")
print("=" * 50)
print(f"📈 Overall Accuracy:")
print(f"   Clear Model: {accuracies[0]:.4f} ({accuracies[0]*100:.2f}%)")
print(f"   FHE Model: {accuracies[1]:.4f} ({accuracies[1]*100:.2f}%)")
print(f"   Difference: {abs(accuracies[0] - accuracies[1]):.4f}")

print(f"\n🤝 Prediction Agreement:")
print(f"   Agreement Rate: {agreement_rate:.4f} ({agreement_rate*100:.2f}%)")
print(f"   Agreeing Predictions: {np.sum(agreement)}/{comparison_samples}")
print(f"   Disagreeing Predictions: {np.sum(~agreement)}/{comparison_samples}")

print(f"\n📋 Class-wise Performance:")
for i, class_name in enumerate(class_names):
    print(f"   {class_name}:")
    print(f"      Clear: {class_accuracies_clear[i]:.3f}")
    print(f"      FHE: {class_accuracies_encrypted[i]:.3f}")
    print(f"      Diff: {abs(class_accuracies_clear[i] - class_accuracies_encrypted[i]):.3f}")

## 8. Confusion Matrix Heatmaps

Display detailed confusion matrices for both clear and encrypted models.

In [ ]:
# Create confusion matrix heatmaps
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Confusion Matrix Comparison: Clear vs FHE Models', fontsize=16, fontweight='bold')

# Calculate confusion matrices
cm_clear = confusion_matrix(comparison_true, comparison_clear)
cm_encrypted = confusion_matrix(comparison_true, comparison_encrypted)

# Create heatmaps
sns.heatmap(cm_clear, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            ax=ax1, cbar_kws={'label': 'Count'})
ax1.set_title(f'Clear Model Confusion Matrix\nAccuracy: {accuracies[0]:.3f}', fontweight='bold')
ax1.set_xlabel('Predicted Label')
ax1.set_ylabel('True Label')

sns.heatmap(cm_encrypted, annot=True, fmt='d', cmap='Reds',
            xticklabels=class_names, yticklabels=class_names,
            ax=ax2, cbar_kws={'label': 'Count'})
ax2.set_title(f'FHE Model Confusion Matrix\nAccuracy: {accuracies[1]:.3f}', fontweight='bold')
ax2.set_xlabel('Predicted Label')
ax2.set_ylabel('True Label')

plt.tight_layout()
plt.show()

# Calculate and display detailed metrics for each class
print("📊 Detailed Performance Metrics by Class:")
print("=" * 70)

from sklearn.metrics import precision_recall_fscore_support

# Calculate metrics for clear model
precision_clear, recall_clear, f1_clear, support_clear = precision_recall_fscore_support(
    comparison_true, comparison_clear, average=None, labels=range(len(class_names))
)

# Calculate metrics for encrypted model
precision_encrypted, recall_encrypted, f1_encrypted, support_encrypted = precision_recall_fscore_support(
    comparison_true, comparison_encrypted, average=None, labels=range(len(class_names))
)

# Create a detailed comparison table
print(f"{'Class':<15} {'Model':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Support':<10}")
print("-" * 70)

for i, class_name in enumerate(class_names):
    print(f"{class_name:<15} {'Clear':<10} {precision_clear[i]:<10.3f} {recall_clear[i]:<10.3f} {f1_clear[i]:<10.3f} {support_clear[i]:<10}")
    print(f"{'':<15} {'FHE':<10} {precision_encrypted[i]:<10.3f} {recall_encrypted[i]:<10.3f} {f1_encrypted[i]:<10.3f} {support_encrypted[i]:<10}")
    print("-" * 70)

# Calculate macro and weighted averages
precision_clear_macro = np.mean(precision_clear)
recall_clear_macro = np.mean(recall_clear)
f1_clear_macro = np.mean(f1_clear)

precision_encrypted_macro = np.mean(precision_encrypted)
recall_encrypted_macro = np.mean(recall_encrypted)
f1_encrypted_macro = np.mean(f1_encrypted)

print(f"{'MACRO AVG':<15} {'Clear':<10} {precision_clear_macro:<10.3f} {recall_clear_macro:<10.3f} {f1_clear_macro:<10.3f} {np.sum(support_clear):<10}")
print(f"{'MACRO AVG':<15} {'FHE':<10} {precision_encrypted_macro:<10.3f} {recall_encrypted_macro:<10.3f} {f1_encrypted_macro:<10.3f} {np.sum(support_encrypted):<10}")

# Additional confusion matrix analysis
print(f"\n🔍 Confusion Matrix Analysis:")
print(f"Clear Model:")
print(f"   True Positives (diagonal): {np.trace(cm_clear)}")
print(f"   Total Predictions: {np.sum(cm_clear)}")
print(f"   Correct Predictions: {np.trace(cm_clear)}/{np.sum(cm_clear)}")

print(f"FHE Model:")
print(f"   True Positives (diagonal): {np.trace(cm_encrypted)}")
print(f"   Total Predictions: {np.sum(cm_encrypted)}")
print(f"   Correct Predictions: {np.trace(cm_encrypted)}/{np.sum(cm_encrypted)}")

# Calculate per-class error analysis
print(f"\n❌ Error Analysis:")
for i, class_name in enumerate(class_names):
    clear_errors = np.sum(cm_clear[i, :]) - cm_clear[i, i]
    encrypted_errors = np.sum(cm_encrypted[i, :]) - cm_encrypted[i, i]
    print(f"   {class_name}: Clear={clear_errors} errors, FHE={encrypted_errors} errors")

## 9. Performance and Privacy Analysis

Analyze the trade-offs between performance, privacy, and computational cost.

In [ ]:
# Performance and Privacy Analysis
print("🔍 Performance and Privacy Analysis")
print("=" * 60)

# Create performance comparison visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('FHE Performance and Privacy Analysis', fontsize=16, fontweight='bold')

# 1. Training Time Comparison
training_times = [clear_training_time, fhe_training_time]
training_labels = ['Clear Model', 'FHE Model']

bars = ax1.bar(training_labels, training_times, color=['skyblue', 'lightcoral'], alpha=0.8)
ax1.set_title('Training Time Comparison', fontweight='bold')
ax1.set_ylabel('Time (seconds)')
ax1.grid(True, alpha=0.3, axis='y')

for bar, time_val in zip(bars, training_times):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(training_times)*0.01,
             f'{time_val:.2f}s', ha='center', va='bottom', fontweight='bold')

# 2. Inference Time Comparison
if ENCRYPTED_INFERENCE_SUCCESS:
    inference_times = [clear_inference_time/len(X_test), fhe_inference_time/n_encrypted_samples]
    inference_labels = ['Clear Model\n(per sample)', 'FHE Model\n(per sample)']
else:
    inference_times = [clear_inference_time/len(X_test), fhe_clear_inference_time/len(X_test)]
    inference_labels = ['Clear Model\n(per sample)', 'FHE Model\n(simulation, per sample)']

bars = ax2.bar(inference_labels, inference_times, color=['lightgreen', 'orange'], alpha=0.8)
ax2.set_title('Inference Time Comparison', fontweight='bold')
ax2.set_ylabel('Time per Sample (seconds)')
ax2.grid(True, alpha=0.3, axis='y')

for bar, time_val in zip(bars, inference_times):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(inference_times)*0.01,
             f'{time_val:.4f}s', ha='center', va='bottom', fontweight='bold')

# 3. Privacy vs Utility Trade-off
privacy_levels = ['No Privacy\n(Clear)', 'Full Privacy\n(FHE)']
utility_scores = [accuracies[0], accuracies[1]]
privacy_scores = [0, 100]  # Privacy score (0 = no privacy, 100 = full privacy)

# Create dual-axis plot
ax3_twin = ax3.twinx()

line1 = ax3.plot(privacy_levels, utility_scores, 'bo-', linewidth=2, markersize=8, label='Utility (Accuracy)')
line2 = ax3_twin.plot(privacy_levels, privacy_scores, 'ro-', linewidth=2, markersize=8, label='Privacy Level')

ax3.set_title('Privacy vs Utility Trade-off', fontweight='bold')
ax3.set_ylabel('Accuracy', color='blue')
ax3_twin.set_ylabel('Privacy Level (%)', color='red')
ax3.tick_params(axis='y', labelcolor='blue')
ax3_twin.tick_params(axis='y', labelcolor='red')
ax3.grid(True, alpha=0.3)

# Add value labels
for i, (util, priv) in enumerate(zip(utility_scores, privacy_scores)):
    ax3.text(i, util + 0.01, f'{util:.3f}', ha='center', va='bottom', color='blue', fontweight='bold')
    ax3_twin.text(i, priv + 2, f'{priv}%', ha='center', va='bottom', color='red', fontweight='bold')

# 4. Model Complexity Comparison
complexity_metrics = ['Features', 'Parameters', 'Model Size (KB)']

# Estimate model complexity (simplified)
n_features = X_train.shape[1]
n_classes = len(class_names)
clear_params = n_features * n_classes + n_classes  # weights + biases
fhe_params = clear_params  # Same model architecture

# Estimate model sizes (simplified)
clear_size = clear_params * 8 / 1024  # 8 bytes per parameter, convert to KB
fhe_size = clear_params * 8 / 1024 + 10  # Additional overhead for FHE

complexity_clear = [n_features, clear_params, clear_size]
complexity_fhe = [n_features, fhe_params, fhe_size]

x_pos = np.arange(len(complexity_metrics))
width = 0.35

bars1 = ax4.bar(x_pos - width/2, complexity_clear, width, label='Clear Model', color='skyblue', alpha=0.8)
bars2 = ax4.bar(x_pos + width/2, complexity_fhe, width, label='FHE Model', color='lightcoral', alpha=0.8)

ax4.set_title('Model Complexity Comparison', fontweight='bold')
ax4.set_xlabel('Complexity Metrics')
ax4.set_ylabel('Count / Size')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(complexity_metrics)
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print comprehensive analysis
print("\n📊 Comprehensive Performance Analysis:")
print("-" * 60)

print(f"🏃 Training Performance:")
print(f"   Clear Model: {clear_training_time:.2f} seconds")
print(f"   FHE Model: {fhe_training_time:.2f} seconds")
print(f"   Overhead: {(fhe_training_time/clear_training_time - 1)*100:.1f}% slower")

print(f"\n⚡ Inference Performance:")
if ENCRYPTED_INFERENCE_SUCCESS:
    speedup_factor = (fhe_inference_time/n_encrypted_samples) / (clear_inference_time/len(X_test))
    print(f"   Clear Model: {clear_inference_time/len(X_test):.6f} seconds/sample")
    print(f"   FHE Model: {fhe_inference_time/n_encrypted_samples:.6f} seconds/sample")
    print(f"   Overhead: {speedup_factor:.0f}x slower")
else:
    print(f"   Clear Model: {clear_inference_time/len(X_test):.6f} seconds/sample")
    print(f"   FHE Model (sim): {fhe_clear_inference_time/len(X_test):.6f} seconds/sample")

print(f"\n🎯 Accuracy Analysis:")
print(f"   Clear Model: {accuracies[0]:.4f}")
print(f"   FHE Model: {accuracies[1]:.4f}")
print(f"   Accuracy Loss: {(accuracies[0] - accuracies[1]):.4f} ({((accuracies[0] - accuracies[1])/accuracies[0]*100):.2f}%)")

print(f"\n🔒 Privacy Benefits:")
print(f"   Data Privacy: {'✅ Full encryption' if ENCRYPTED_INFERENCE_SUCCESS else '⚠️ Simulation mode'}")
print(f"   Model Privacy: {'✅ Encrypted inference' if ENCRYPTED_INFERENCE_SUCCESS else '⚠️ Clear inference'}")
print(f"   Compliance: {'✅ GDPR/HIPAA compatible' if ENCRYPTED_INFERENCE_SUCCESS else '⚠️ Limited compliance'}")

print(f"\n💰 Cost-Benefit Analysis:")
print(f"   Accuracy Trade-off: {abs(accuracies[0] - accuracies[1]):.4f} accuracy loss")
print(f"   Privacy Gain: {'100% data privacy' if ENCRYPTED_INFERENCE_SUCCESS else 'Simulation only'}")
print(f"   Computational Cost: {'High (true FHE)' if ENCRYPTED_INFERENCE_SUCCESS else 'Low (simulation)'}")
print(f"   Recommendation: {'Suitable for high-privacy applications' if ENCRYPTED_INFERENCE_SUCCESS else 'Good for development/testing'}")

## 10. Summary and Conclusions

Summary of the FHE training demonstration and key insights.

In [ ]:
# Export results and generate summary
output_dir = Path('../data/results')
output_dir.mkdir(parents=True, exist_ok=True)

# Save model comparison results
results_summary = {
    "experiment_info": {
        "timestamp": pd.Timestamp.now().isoformat(),
        "dataset_size": len(df),
        "features": X_train.shape[1],
        "classes": len(class_names),
        "test_samples": comparison_samples,
        "fhe_mode": "encrypted" if ENCRYPTED_INFERENCE_SUCCESS else "simulation"
    },
    "model_performance": {
        "clear_model": {
            "accuracy": float(accuracies[0]),
            "training_time": float(clear_training_time),
            "inference_time_per_sample": float(clear_inference_time/len(X_test))
        },
        "fhe_model": {
            "accuracy": float(accuracies[1]),
            "training_time": float(fhe_training_time),
            "inference_time_per_sample": float(fhe_inference_time/n_encrypted_samples if ENCRYPTED_INFERENCE_SUCCESS else fhe_clear_inference_time/len(X_test)),
            "compilation_time": float(compilation_time)
        }
    },
    "comparison_metrics": {
        "accuracy_difference": float(abs(accuracies[0] - accuracies[1])),
        "prediction_agreement_rate": float(agreement_rate),
        "privacy_level": "high" if ENCRYPTED_INFERENCE_SUCCESS else "simulation"
    }
}

# Save results to JSON
results_file = output_dir / 'fhe_training_demo_results.json'
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(results_summary, f, indent=2)

print("💾 Results saved to:", results_file)

# Generate comprehensive summary
print("\n🎉 FHE Training Demo Summary")
print("=" * 60)

print(f"📊 Dataset Information:")
print(f"   Total Records: {len(df)}")
print(f"   Features: {X_train.shape[1]}")
print(f"   Classes: {len(class_names)} ({', '.join(class_names)})")
print(f"   Train/Test Split: {len(X_train)}/{len(X_test)}")

print(f"\n🤖 Model Training Results:")
print(f"   Clear Model Accuracy: {accuracies[0]:.4f} ({accuracies[0]*100:.2f}%)")
print(f"   FHE Model Accuracy: {accuracies[1]:.4f} ({accuracies[1]*100:.2f}%)")
print(f"   Accuracy Difference: {abs(accuracies[0] - accuracies[1]):.4f}")
print(f"   Prediction Agreement: {agreement_rate:.4f} ({agreement_rate*100:.2f}%)")

print(f"\n⚡ Performance Metrics:")
print(f"   Clear Training Time: {clear_training_time:.2f} seconds")
print(f"   FHE Training Time: {fhe_training_time:.2f} seconds")
if ENCRYPTED_INFERENCE_SUCCESS:
    print(f"   FHE Compilation Time: {compilation_time:.2f} seconds")
    print(f"   Encrypted Inference: {fhe_inference_time:.2f} seconds ({n_encrypted_samples} samples)")
    print(f"   Encryption Overhead: {(fhe_inference_time/n_encrypted_samples)/(clear_inference_time/len(X_test)):.0f}x slower")

print(f"\n🔒 Privacy Analysis:")
print(f"   FHE Mode: {'✅ True Encryption' if ENCRYPTED_INFERENCE_SUCCESS else '⚠️ Simulation'}")
print(f"   Data Privacy: {'✅ Fully Protected' if ENCRYPTED_INFERENCE_SUCCESS else '⚠️ Not Encrypted'}")
print(f"   Model Privacy: {'✅ Encrypted Inference' if ENCRYPTED_INFERENCE_SUCCESS else '⚠️ Clear Inference'}")

print(f"\n📈 Key Insights:")
insights = []
if abs(accuracies[0] - accuracies[1]) < 0.05:
    insights.append("✅ FHE model maintains high accuracy compared to clear model")
else:
    insights.append("⚠️ FHE model shows some accuracy degradation")

if agreement_rate > 0.8:
    insights.append("✅ High agreement between clear and FHE predictions")
else:
    insights.append("⚠️ Moderate agreement between clear and FHE predictions")

if ENCRYPTED_INFERENCE_SUCCESS:
    insights.append("✅ Successfully demonstrated true FHE encrypted inference")
    insights.append("✅ Suitable for privacy-critical medical applications")
else:
    insights.append("⚠️ Demonstrated FHE simulation mode")
    insights.append("📝 Consider installing Concrete-ML for true FHE capabilities")

for insight in insights:
    print(f"   {insight}")

print(f"\n🔄 Next Steps:")
print("1. 🔧 Optimize FHE parameters for better performance")
print("2. 📊 Evaluate on larger datasets")
print("3. 🛡️ Implement privacy-preserving SVM comparison")
print("4. 📈 Generate comprehensive evaluation reports")
print("5. 🔒 Conduct security analysis of FHE implementation")

print(f"\n✨ FHE Training Demo completed successfully!")
print(f"📁 Results saved to: {results_file}")

# Display final model comparison table
print(f"\n📋 Final Model Comparison:")
print("-" * 60)
print(f"{'Metric':<25} {'Clear Model':<15} {'FHE Model':<15} {'Difference':<15}")
print("-" * 60)
print(f"{'Accuracy':<25} {accuracies[0]:<15.4f} {accuracies[1]:<15.4f} {abs(accuracies[0] - accuracies[1]):<15.4f}")
print(f"{'Training Time (s)':<25} {clear_training_time:<15.2f} {fhe_training_time:<15.2f} {abs(clear_training_time - fhe_training_time):<15.2f}")
if ENCRYPTED_INFERENCE_SUCCESS:
    print(f"{'Inference Time (s/sample)':<25} {clear_inference_time/len(X_test):<15.6f} {fhe_inference_time/n_encrypted_samples:<15.6f} {abs((clear_inference_time/len(X_test)) - (fhe_inference_time/n_encrypted_samples)):<15.6f}")
print(f"{'Privacy Level':<25} {'None':<15} {'High' if ENCRYPTED_INFERENCE_SUCCESS else 'Simulation':<15} {'-':<15}")
print("-" * 60)